In [1]:
import os
from os.path import exists

import tensorflow as tf

img_shape = (512,512,3)
BATCH_SIZE = 1
IMG_SIZE = (512, 512)

PATH="E:\\GSoC\\ct_imaging\\ML_MODEL\\dataset"
train_dir=os.path.join(PATH, 'train')
validation_dir=os.path.join(PATH, 'validation')

In [2]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 79 files belonging to 2 classes.
Found 20 files belonging to 2 classes.


In [3]:
base_model = tf.keras.applications.VGG19(input_shape=img_shape, include_top=False, weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0     

In [4]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
history = model.fit( train_dataset, batch_size=100, epochs=50 )

Epoch 1/50
79/79 [==============================] - 89s 1s/step - loss: 0.9363 - accuracy: 0.4051
Epoch 2/50
79/79 [==============================] - 93s 1s/step - loss: 0.8605 - accuracy: 0.4430
Epoch 3/50
79/79 [==============================] - 96s 1s/step - loss: 0.8356 - accuracy: 0.5570
Epoch 4/50
79/79 [==============================] - 96s 1s/step - loss: 0.8207 - accuracy: 0.4937
Epoch 5/50
79/79 [==============================] - 96s 1s/step - loss: 0.8109 - accuracy: 0.5570
Epoch 6/50
79/79 [==============================] - 96s 1s/step - loss: 0.7978 - accuracy: 0.5443
Epoch 7/50
79/79 [==============================] - 96s 1s/step - loss: 0.7865 - accuracy: 0.5823
Epoch 8/50
79/79 [==============================] - 96s 1s/step - loss: 0.7727 - accuracy: 0.5823
Epoch 9/50
79/79 [==============================] - 94s 1s/step - loss: 0.7641 - accuracy: 0.6076
Epoch 10/50
79/79 [==============================] - 94s 1s/step - loss: 0.7542 - accuracy: 0.5949
Epoch 11/50
79/79 [

In [42]:
history

In [43]:
model.save('image-model.h5')

In [44]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

# model.evaluate(train_dataset)

Found 20 files belonging to 2 classes.


In [45]:
model.evaluate(validation_dataset)

20/20 [==============================] - 26s 1s/step - loss: 0.5430 - accuracy: 0.6500


[0.5430178642272949, 0.6499999761581421]

In [47]:
model.predict(validation_dataset)

20/20 [==============================] - 20s 1s/step


array([[0.78846645],
       [0.718188  ],
       [0.846129  ],
       [0.64539707],
       [0.88982904],
       [0.846129  ],
       [0.48921198],
       [0.7658114 ],
       [0.8554429 ],
       [0.8282568 ],
       [0.2374105 ],
       [0.67518985],
       [0.20049445],
       [0.6043703 ],
       [0.20385756],
       [0.87960225],
       [0.492195  ],
       [0.5957947 ],
       [0.833088  ],
       [0.7347689 ]], dtype=float32)

In [5]:
import cv2

tf_model_predictions=tf.keras.models.load_model('image-model.h5')
img = cv2.imread("E:\\GSoC\\ct_imaging\\ML_MODEL\\dataset\\validation\\cancer\\ID_0047_AGE_0069_CONTRAST_1_CT.jpg")
img = cv2.resize(img, (512,512))
img = img.reshape(1,512,512,3)

classes = tf_model_predictions.predict(img)

1/1 [==============================] - 3s 3s/step


In [6]:
classes

array([[0.0256956]], dtype=float32)

In [15]:
import tensorflow_hub as hub
import tensorflow as tf

def predict(image_path):
    model_h5 = tf.keras.models.load_model(
        './image-model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

    img = cv2.imread(image_path)
    img = cv2.resize(img, (512,512))
    img = img.reshape(1,512,512,3)
    pred90 = model_h5.predict(img)  
    val = ''
    pred90 = pred90[0][0]*100
    if pred90 < 90.0:
        val = 'Normal'
    else:
        val = 'Cancer'
    
    return val

In [17]:
val=predict("E:\\GSoC\\ct_imaging\\ML_MODEL\\dataset\\validation\\cancer\\ID_0047_AGE_0069_CONTRAST_1_CT.jpg")

1/1 [==============================] - 1s 1s/step


In [18]:
val

'Normal'